# Surface Super-Resolution (2D) - Demo Notebook

基于 FuXi 技术的表层数据超分项目演示

## 关键特性
- ✨ 2D Swin Transformer 架构
- ✨ 轻量化模型设计
- ✨ 4倍超分（256×256 → 1024×1024）
- ✨ 多种损失函数支持

## 1. 环境设置

In [ ]:
import sys
from pathlib import Path

# 添加项目路径
project_root = Path().cwd()
sys.path.insert(0, str(project_root))

import numpy as np
import mindspore as ms
from mindspore import context, set_seed
import logging

# 设置随机种子
set_seed(42)
np.random.seed(42)

# 设置上下文
context.set_context(mode=context.PYNATIVE_MODE, device_target="CPU")

print("✓ 环境设置完成")
print(f"MindSpore version: {ms.__version__}")

## 2. 导入项目模块

In [ ]:
from src.surface_sr_net import SurfaceSRNet
from src.surface_sr import (
    MAELoss,
    MSELoss,
    CombinedLoss,
    create_loss_fn,
    create_optimizer,
)
from src.data import create_mindspore_dataset, SurfaceDataLoader
from src.eval import Evaluator, Metrics
from src.utils import Timer, setup_logger

print("✓ 所有模块导入成功")

## 3. 生成模拟数据

In [ ]:
# 生成模拟的低分辨率和高分辨率数据
# 表层通常有 6 个主要变量：温度、湿度、压力、风速U、风速V、降水

num_samples = 50
in_channels = 6
low_h, low_w = 256, 256
high_h, high_w = 1024, 1024

# 生成训练数据
train_low = np.random.randn(num_samples, in_channels, low_h, low_w).astype(np.float32)
train_high = np.random.randn(num_samples, in_channels, high_h, high_w).astype(
    np.float32
)

# 生成验证数据
val_low = np.random.randn(10, in_channels, low_h, low_w).astype(np.float32)
val_high = np.random.randn(10, in_channels, high_h, high_w).astype(np.float32)

print(f"✓ 数据生成完成")
print(f"  训练集: LR {train_low.shape}, HR {train_high.shape}")
print(f"  验证集: LR {val_low.shape}, HR {val_high.shape}")

## 4. 构建模型

In [ ]:
# 构建表层超分网络
model = SurfaceSRNet(
    in_channels=6,
    out_channels=6,
    low_h=256,
    low_w=256,
    high_h=1024,
    high_w=1024,
    embed_dim=96,
    depths=12,
    num_heads=8,
    kernel_size=(4, 4),
    batch_size=1,
)

print("✓ 模型构建完成")
print(f"  模型: SurfaceSRNet")
print(f"  嵌入维度: 96")
print(f"  Swin Block 数: 12")
print(f"  超分倍数: 4x")

## 5. 模型推理演示

In [ ]:
# 单个样本推理演示
sample_input = ms.Tensor(train_low[:1], ms.float32)

timer = Timer()
timer.start("inference")

with ms.no_grad():
    output = model(sample_input)

elapsed = timer.stop("inference")
output_np = output.asnumpy()

print("✓ 推理完成")
print(f"  输入形状: {sample_input.shape}")
print(f"  输出形状: {output.shape}")
print(f"  推理时间: {elapsed:.4f}s")
print(f"  输出统计:")
print(f"    范围: [{output_np.min():.4f}, {output_np.max():.4f}]")
print(f"    均值: {output_np.mean():.4f}")
print(f"    标准差: {output_np.std():.4f}")

## 6. 损失函数演示

In [ ]:
# 创建不同的损失函数
mae_loss = MAELoss()
mse_loss = MSELoss()
combined_loss = CombinedLoss(mae_weight=0.7, mse_weight=0.3)

# 计算损失
pred = ms.Tensor(train_high[:1], ms.float32)
target = ms.Tensor(train_high[:1], ms.float32)

mae = mae_loss(pred, target)
mse = mse_loss(pred, target)
combined = combined_loss(pred, target)

print("✓ 损失函数演示")
print(f"  MAE Loss: {mae.item():.6f}")
print(f"  MSE Loss: {mse.item():.6f}")
print(f"  Combined Loss: {combined.item():.6f}")

## 7. 评估指标演示

In [ ]:
# 创建评估器
evaluator = Evaluator(metrics_list=["mae", "mse", "rmse", "psnr", "ssim"])

# 模拟预测和目标
pred_sample = output_np[0]  # (6, 1024, 1024)
target_sample = train_high[0]  # (6, 1024, 1024)

# 计算评估指标
results = evaluator.evaluate(pred_sample, target_sample)

print("✓ 评估指标")
for metric, value in results.items():
    if metric == "psnr":
        print(f"  {metric.upper()}: {value:.2f} dB")
    elif metric == "ssim":
        print(f"  {metric.upper()}: {value:.4f}")
    else:
        print(f"  {metric.upper()}: {value:.6f}")

## 8. 模型架构分析

In [ ]:
# 统计参数数量
total_params = 0
for param in model.trainable_params():
    total_params += np.prod(param.shape)

print("✓ 模型架构分析")
print(f"  总参数数: {total_params:,}")
print(f"  模型大小: ~{total_params * 4 / (1024**2):.2f} MB (float32)")

# 显示模型主要组件
print(f"\n  主要组件:")
print(f"    - SurfaceEmbed (2D Patch Embedding)")
print(f"    - DownSample2D (下采样)")
print(f"    - SurfaceSwinBlock×12 (Swin Transformer)")
print(f"    - UpSample2D (上采样)")
print(f"    - PatchRecover2D (Patch恢复到高分辨率)")

## 9. 数据标准化演示

In [ ]:
# 数据标准化
normalized_data, mean, std = SurfaceDataLoader.normalize(train_low)

print("✓ 数据标准化")
print(f"  原始数据:")
print(f"    范围: [{train_low.min():.4f}, {train_low.max():.4f}]")
print(f"    均值: {train_low.mean():.4f}")
print(f"    标准差: {train_low.std():.4f}")

print(f"\n  标准化后:")
print(f"    范围: [{normalized_data.min():.4f}, {normalized_data.max():.4f}]")
print(f"    均值: {normalized_data.mean():.4f}")
print(f"    标准差: {normalized_data.std():.4f}")

# 反标准化
denormalized = SurfaceDataLoader.denormalize(normalized_data, mean, std)
print(f"\n  反标准化误差: {np.mean(np.abs(denormalized - train_low)):.6e}")

## 10. 批量推理演示

In [ ]:
# 批量推理
batch_size = 4
batch_input = ms.Tensor(train_low[:batch_size], ms.float32)

timer.start("batch_inference")

with ms.no_grad():
    batch_output = model(batch_input)

elapsed_batch = timer.stop("batch_inference")

print("✓ 批量推理演示")
print(f"  批量大小: {batch_size}")
print(f"  总推理时间: {elapsed_batch:.4f}s")
print(f"  平均每样本时间: {elapsed_batch/batch_size:.4f}s")
print(f"  吞吐量: {batch_size/elapsed_batch:.2f} samples/s")

## 11. 配置文件示例

In [ ]:
from src.config import load_yaml_config
from pathlib import Path

config_path = Path("configs/surface_sr.yaml")
if config_path.exists():
    config = load_yaml_config(str(config_path))
    print("✓ 配置文件内容:")
    print(f"\n  模型配置:")
    print(f"    输入通道数: {config.get('model', {}).get('in_channels')}")
    print(f"    输出通道数: {config.get('model', {}).get('out_channels')}")
    print(f"    Swin Block 数: {config.get('model', {}).get('depths')}")
else:
    print("⚠ 配置文件不存在")

## 12. 总结

In [ ]:
print("=" * 60)
print("Surface Super-Resolution (2D) - 项目总结")
print("=" * 60)
print()
print("✓ 核心特性:")
print("  1. 基于 Swin Transformer 的 2D 超分网络")
print("  2. 4 倍超分（256×256 → 1024×1024）")
print("  3. 多种损失函数支持（MAE, MSE, Combined）")
print("  4. 灵活的数据处理和评估")
print()
print("✓ 快速使用:")
print("  python main.py --config_file_path ./configs/surface_sr.yaml")
print()
print("✓ 项目结构:")
print("  - src/: 核心代码")
print("  - configs/: 配置文件")
print("  - scripts/: 训练和评估脚本")
print("  - mindearth/: MindEarth 依赖")
print()
print("✓ 下一步:")
print("  1. 准备自己的数据")
print("  2. 修改配置文件")
print("  3. 开始训练: bash scripts/run_standalone_train.sh")
print()
print("=" * 60)